In [ ]:
# This notebook is used to create a map which is colored based on how many tweets we downloaded for each country
! pip install geopandas
! pip install pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.3 MB 34.5 MB/s 
     |████████████████████████████████| 16.7 MB 688 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=9f34bd4f35d4f2875d54e8f3aa9fc8a6df8a60226f4ef2fbdeaadeda0d68f8c3
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [ ]:
# Get data
! rm -rf covid_data
! git clone https://github.com/PatrickNiccolai2/covid_data

Cloning into 'covid_data'...
remote: Enumerating objects: 2033, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 2033 (delta 22), reused 82 (delta 19), pack-reused 1948
Receiving objects: 100% (2033/2033), 270.54 MiB | 24.75 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [ ]:
import matplotlib.pyplot as plt
import geopandas
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import folium
import os
import pycountry

In [ ]:
# Unzip data
!unzip /content/covid_data/tweets_with_sent -d /content/covid_data/unzip_tweets_with_sent

Streaming output truncated to the last 5000 lines.
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_118805.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_65049.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_43930.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_112496.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_123893.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_72535.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_6089.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_34560.txt  
  inflating: /content/covid_data/unzip_tweets_with_sent/tweets_with_sent/sent_shuffled_tweet_45399.txt  
  

In [ ]:
# Read data into array
all_files_text = []
for filename in os.listdir("/content/covid_data/unzip_tweets_with_sent/tweets_with_sent"):
  if(filename[0] == "s"):
    file_path = "/content/covid_data/unzip_tweets_with_sent/tweets_with_sent/" + filename
    file = open(file_path, "r")
    lines = file.read()
    lines_arr = lines.split("\n\n")
    sent = lines_arr[-1]
    id = lines_arr[-2]
    place = lines_arr[-3]
    coords = lines_arr[-4]
    time = lines_arr[-5]

    tweet_text = ""
    for i in range(5, len(lines_arr) + 1):
      tweet_text = tweet_text + lines_arr[-i]

    if("johnson" not in tweet_text.lower()):
      full_tweet = [tweet_text, time, coords, place, id, sent]
      all_files_text.append(full_tweet)
    file.close()


In [ ]:
# Create dataframe
all_tweets_df = pd.DataFrame(all_files_text, columns=["tweet_text", "time", "coords", "place", "tweet_id", "sentiment"])

In [ ]:
print(all_files_text[0][3])

Place(_api=<tweepy.api.API object at 0x7fe56c595af0>, id='4ec01c9dbc693497', url='https://api.twitter.com/1.1/geo/id/4ec01c9dbc693497.json', place_type='admin', name='Florida', full_name='Florida, USA', country_code='US', country='United States', contained_within=[], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x7fe56c595af0>, type='Polygon', coordinates=[[[-87.634643, 24.396308], [-79.974307, 24.396308], [-79.974307, 31.001056], [-87.634643, 31.001056]]]), attributes={})


In [ ]:
# Parce place string to get country code
def get_country(place_str):
  cc_loc = place_str.find("country_code")
  code_loc = cc_loc + 14
  code_str = place_str[code_loc:code_loc + 2]
  return code_str

all_tweets_df["country"] = all_tweets_df["place"].apply(get_country)

In [ ]:
print(all_tweets_df.head(10))

                                          tweet_text  \
0  2021-04-29 13:09:32+00:00@HellOnHeels2020 It w...   
1  2021-03-25 01:13:10+00:00When NYers needed vac...   
2  2021-07-02 18:42:54+00:00Checkout @watvaxtrack...   
3  2021-06-02 08:41:02+00:00Completely agree with...   
4  2021-03-16 11:00:30+00:00@MajorDMalpas Delays ...   
5  2021-04-07 21:00:50+00:00For #WorldHealthDay i...   
6  2021-03-06 05:24:03+00:00So... #toronto has be...   
7  2021-08-28 17:44:57+00:00https://t.co/iyNARb3Y...   
8  2021-06-13 23:47:01+00:00@chhardman Well if yo...   
9  2021-04-01 06:47:59+00:00ZERO long term trials...   

                        time coords  \
0  2021-04-29 13:09:32+00:00   None   
1  2021-03-25 01:13:10+00:00   None   
2  2021-07-02 18:42:54+00:00   None   
3  2021-06-02 08:41:02+00:00   None   
4  2021-03-16 11:00:30+00:00   None   
5  2021-04-07 21:00:50+00:00   None   
6  2021-03-06 05:24:03+00:00   None   
7  2021-08-28 17:44:57+00:00   None   
8  2021-06-13 23:47:01+00:00   N

In [ ]:
# Convert 2 letter country code to 3 letter country code
def alpha2_to_iso3(alpha2):
  for country in pycountry.countries:
    if country.alpha_2 == alpha2:
      return country.alpha_3
  return ""

all_tweets_df["country3"] = all_tweets_df["country"].apply(alpha2_to_iso3)

In [ ]:
print(all_tweets_df.head(10))

                                          tweet_text  \
0  2021-04-29 13:09:32+00:00@HellOnHeels2020 It w...   
1  2021-03-25 01:13:10+00:00When NYers needed vac...   
2  2021-07-02 18:42:54+00:00Checkout @watvaxtrack...   
3  2021-06-02 08:41:02+00:00Completely agree with...   
4  2021-03-16 11:00:30+00:00@MajorDMalpas Delays ...   
5  2021-04-07 21:00:50+00:00For #WorldHealthDay i...   
6  2021-03-06 05:24:03+00:00So... #toronto has be...   
7  2021-08-28 17:44:57+00:00https://t.co/iyNARb3Y...   
8  2021-06-13 23:47:01+00:00@chhardman Well if yo...   
9  2021-04-01 06:47:59+00:00ZERO long term trials...   

                        time coords  \
0  2021-04-29 13:09:32+00:00   None   
1  2021-03-25 01:13:10+00:00   None   
2  2021-07-02 18:42:54+00:00   None   
3  2021-06-02 08:41:02+00:00   None   
4  2021-03-16 11:00:30+00:00   None   
5  2021-04-07 21:00:50+00:00   None   
6  2021-03-06 05:24:03+00:00   None   
7  2021-08-28 17:44:57+00:00   None   
8  2021-06-13 23:47:01+00:00   N

In [ ]:
# Group to get number of tweets per country
samples_per_country_df = all_tweets_df.groupby(["country3"]).count()

In [ ]:
print(samples_per_country_df.head(10))

          tweet_text  time  coords  place  tweet_id  sentiment  country
country3                                                               
                 218   218     218    218       218        218      218
ABW                5     5       5      5         5          5        5
AFG                9     9       9      9         9          9        9
AGO                1     1       1      1         1          1        1
AIA                3     3       3      3         3          3        3
ALB               10    10      10     10        10         10       10
AND                2     2       2      2         2          2        2
ARE              296   296     296    296       296        296      296
ARG              375   375     375    375       375        375      375
ARM                6     6       6      6         6          6        6


In [ ]:
# Read in dataframe from geopandas which has country location for mapping
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# fix for bug https://github.com/geopandas/geopandas/issues/1041
world.loc[world['name'] == 'France', 'iso_a3'] = 'FRA'
world.loc[world['name'] == 'Norway', 'iso_a3'] = 'NOR'
world.loc[world['name'] == 'N. Cyprus', 'iso_a3'] = 'CYP'
world.loc[world['name'] == 'Somaliland', 'iso_a3'] = 'SOM'
world.loc[world['name'] == 'Kosovo', 'iso_a3'] = 'RKS'
print(world.head(10))

     pop_est      continent                      name iso_a3  gdp_md_est  \
0     920938        Oceania                      Fiji    FJI      8374.0   
1   53950935         Africa                  Tanzania    TZA    150600.0   
2     603253         Africa                 W. Sahara    ESH       906.5   
3   35623680  North America                    Canada    CAN   1674000.0   
4  326625791  North America  United States of America    USA  18560000.0   
5   18556698           Asia                Kazakhstan    KAZ    460700.0   
6   29748859           Asia                Uzbekistan    UZB    202300.0   
7    6909701        Oceania          Papua New Guinea    PNG     28020.0   
8  260580739           Asia                 Indonesia    IDN   3028000.0   
9   44293293  South America                 Argentina    ARG    879400.0   

                                            geometry  
0  MULTIPOLYGON (((180.00000 -16.06713, 180.00000...  
1  POLYGON ((33.90371 -0.95000, 34.07262 -1.05982... 

In [ ]:
# Combine the two dataframes
samples_per_country_df = world.merge(samples_per_country_df, how="left", left_on=['iso_a3'], right_on=['country3'])

In [ ]:
samples_per_country_df["Country"] = samples_per_country_df["name"]

In [ ]:
# Create map with colors based on number of tweets
# The map is saved as an html file
my_map = folium.Map()
folium.Choropleth(
    geo_data=samples_per_country_df,
    name='choropleth',
    data=samples_per_country_df,
    columns=['Country', 'tweet_text'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Tweets'
).add_to(my_map)
my_map.save('tweets_per_country.html')
